# Toxicity Target Type Identification

![SageMaker](https://img.shields.io/badge/SageMaker-%23FF9900.svg?style=for-the-badge&logo=amazon-aws&logoColor=white)

This notebook is a part of [ToChiquinho](https://dougtrajano.github.io/ToChiquinho/) project, which trains a model to classify the type of a given targeted toxic comment.

We used the [OLID-BR](https://dougtrajano.github.io/olid-br/) dataset as the training data. The possible values are:

- `IND`: Individual
- `GRP`: Group
- `OTH`: Other

The model is trained using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).

- [Setup](#Setup)
- [Prepare the data](#Prepare-the-data)
  - [Uploading the data to S3](#Uploading-the-data-to-S3)
- [Training process](#Training-process)
  - [Define the estimator](#Define-the-estimator)
  - [Hyperparameter tuning](#Hyperparameter-tuning)
  - [Training best model](#Training-best-model)
- [Documentation](#Documentation)

## Setup

In this section, we will import the necessary libraries and set up the environment.

In the next cell, you can change the parameters to fit your needs.

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
import os
from ml.arguments import NotebookArguments
from ml.utils import remove_checkpoints

params = NotebookArguments(
    mlflow_experiment_name="toxicity-target-type-identification",
    mlflow_tags={
        "project": "ToChiquinho",
        "dataset": "OLID-BR",
        "model_type": "bert",
        "problem_type": "multi_class_classification"
    },
    sagemaker_execution_role_arn=os.environ.get("SAGEMAKER_EXECUTION_ROLE_ARN"),
    sagemaker_tuning_job_name = "pytorch-training-221209-0011",
    aws_profile_name=os.environ.get("AWS_PROFILE")
)

params

NotebookArguments(num_train_epochs=30, early_stopping_patience=2, batch_size=8, validation_split=0.2, seed=1993, mlflow_experiment_name='toxicity-target-type-identification', mlflow_tags='{"project": "ToChiquinho", "dataset": "OLID-BR", "model_type": "bert", "problem_type": "multi_class_classification"}', sagemaker_image_uri='215993976552.dkr.ecr.us-east-1.amazonaws.com/sagemaker-transformers:1.12.0-gpu-py38')

In [3]:
import boto3
import sagemaker

sagemaker_session = sagemaker.Session(
    boto_session=boto3.Session(profile_name=params.aws_profile_name)
)

bucket_name = sagemaker_session.default_bucket()
prefix = f"ToChiquinho/{params.mlflow_experiment_name}"

if params.sagemaker_execution_role_arn is None:
    params.sagemaker_execution_role_arn = sagemaker.get_execution_role(sagemaker_session)

## Prepare the data

In this section, we will prepare the data to be used in the training process.

We will download OLID-BR dataset from [HuggingFace Datasets](https://huggingface.co/datasets/olidbr), process it and upload it to S3 to be used in the training process.

In [15]:
from datasets import load_dataset

dataset = load_dataset("dougtrajano/olid-br")

  0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
import datasets
from typing import Union

def prepare_dataset(
    dataset: Union[datasets.Dataset, datasets.DatasetDict],
    test_size: float = 0.2,
    seed: int = 42
) -> Union[datasets.Dataset, datasets.DatasetDict]:

    # Filter only rows with is_offensive = "OFF" and is_targeted = "TIN"
    dataset = dataset.filter(
        lambda example: example["is_offensive"] == "OFF" 
        and example["is_targeted"] == "TIN"
        and example["targeted_type"] is not None
    )

    # Filter only "text" and "is_targeted" columns
    dataset = dataset.remove_columns(
        [
            col for col in dataset["train"].column_names if col not in ["text", "targeted_type"]
        ]
    )
    
    dataset = dataset.rename_column("targeted_type", "label")

    # Replace "TIN": 1 and "UNT": 0
    def replace_labels(example):
        if example["label"] == "IND":
            example["label"] = 0
        elif example["label"] == "GRP":
            example["label"] = 1
        elif example["label"] == "OTH":
            example["label"] = 2
        else:
            raise ValueError(f"Invalid label: {example['label']}")
        return example

    dataset = dataset.map(replace_labels)

    train_dataset = dataset["train"].train_test_split(
        test_size=test_size,
        shuffle=True,
        seed=seed
    )

    dataset["train"] = train_dataset["train"]
    dataset["validation"] = train_dataset["test"]

    return dataset


dataset = prepare_dataset(
    dataset,
    test_size=params.validation_split,
    seed=params.seed
)

dataset

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/946 [00:00<?, ?ex/s]

  0%|          | 0/2837 [00:00<?, ?ex/s]

DatasetDict({
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 946
    })
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2269
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 568
    })
})

In [31]:
dataset.save_to_disk("data")

Saving the dataset (0/1 shards):   0%|          | 0/946 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/3 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/2269 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/568 [00:00<?, ? examples/s]

### Uploading the data to S3

We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location.

The return value inputs identifies the location -- we will use later when we start the training job.

In [4]:
# inputs = sagemaker_session.upload_data(
#     path="data",
#     bucket=bucket_name,
#     key_prefix=f"{prefix}/data"
# )

inputs = "s3://sagemaker-us-east-1-215993976552/ToChiquinho/toxicity-target-type-identification/data"

print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-215993976552/ToChiquinho/toxicity-target-type-identification/data


In [17]:
import shutil
shutil.rmtree("data")

## Training session

In this section, we will run the training process.

To use Amazon SageMaker to run Docker containers, we need to provide a Python script for the container to run. In our case, all the code is in the `ml` folder, including the `train.py` script.

We will start doing a hyperparameter tuning process to find the best hyperparameters for our model.

Then, we will train the model using the best hyperparameters found.

In [5]:
import os
import mlflow

mlflow.start_run()

print(f"MLFlow run ID: {mlflow.active_run().info.run_id}")

MLFlow run ID: 1d929fa0d01749f6beb779b8fa72393c


### Define the estimator

We will use the `sagemaker.huggingface.HuggingFace` class to define our estimator.

In [6]:
import logging
from sagemaker.huggingface import HuggingFace

checkpoint_s3_uri = f"s3://{bucket_name}/{prefix}/checkpoints"

instance_type = "ml.g4dn.xlarge" # 4 vCPUs, 16 GB RAM, 1 x NVIDIA T4 16GB GPU - $ 0.736 per hour
# instance_type = "ml.g4dn.2xlarge" # 8 vCPUs, 32 GB RAM, 1 x NVIDIA T4 16GB GPU - $ 0.94 per hour
# instance_type = "ml.g5.xlarge" # 4 vCPUs, 16 GB RAM, 1 x NVIDIA A10G 24GB GPU - $ 1.408 per hour
# instance_type = "ml.g5.2xlarge" # 8 vCPUs, 32 GB RAM, 1 x NVIDIA A10G 24GB GPU - $ 1.515 per hour
# instance_type = "ml.g5.4xlarge" # 16 vCPUs, 64 GB RAM, 2 x NVIDIA A10G 24GB GPU - $ 2.03 per hour
# instance_type = "ml.g5.8xlarge" # 32 vCPUs, 128 GB RAM, 4 x NVIDIA A10G 24GB GPU - $ 3.06 per hour

estimator = HuggingFace(
    entry_point="train.py",
    source_dir="ml",
    base_job_name=params.mlflow_experiment_name,
    container_log_level=logging.DEBUG,
    role=params.sagemaker_execution_role_arn,
    sagemaker_session=sagemaker_session,
    py_version="py38",
    pytorch_version="1.10.2",
    transformers_version="4.17.0",
    instance_count=1,
    instance_type=instance_type,
    use_spot_instances=True,
    max_wait=10800,
    max_run=10800,
    checkpoint_s3_uri=checkpoint_s3_uri,
    checkpoint_local_path="/opt/ml/checkpoints",
    environment={
        "MLFLOW_TRACKING_URI": params.mlflow_tracking_uri,
        "MLFLOW_EXPERIMENT_NAME": params.mlflow_experiment_name,
        "MLFLOW_TRACKING_USERNAME": params.mlflow_tracking_username,
        "MLFLOW_TRACKING_PASSWORD": params.mlflow_tracking_password,
        "MLFLOW_TAGS": params.mlflow_tags,
        "MLFLOW_RUN_ID": mlflow.active_run().info.run_id,
        "MLFLOW_FLATTEN_PARAMS": "True"
    },
    hyperparameters={
        ## If you want to test the code, uncomment the following lines to use smaller datasets
        # "max_train_samples": 100,
        # "max_val_samples": 100,
        # "max_test_samples": 100,
        "num_train_epochs": params.num_train_epochs,
        "early_stopping_patience": params.early_stopping_patience,
        "eval_dataset": "validation",
        "batch_size": params.batch_size,
        "seed": params.seed
    },
)

In [7]:
mlflow.log_params(
    {
        "instance_type": estimator.instance_type,
        "instance_count": estimator.instance_count,
        "early_stopping_patience": params.early_stopping_patience
    }
)

To test our training job before hyperparameter tuning, we will run it with a small number of samples.

In [10]:
estimator.fit(inputs, wait=False)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.estimator:SMDebug Does Not Currently Support                         Distributed Training Jobs With Checkpointing Enabled
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-02-07-00-50-33-262


### Hyperparameter Tuning

We will use the `sagemaker.tuner.HyperparameterTuner` class to run a hyperparameter tuning process.

We use MLflow to track the training process, so we can analyze the results through the MLflow UI.

In [11]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

estimator._hyperparameters.pop("max_train_samples", None)
estimator._hyperparameters.pop("max_val_samples", None)
estimator._hyperparameters.pop("max_test_samples", None)

tuner = HyperparameterTuner(
    estimator,
    max_jobs=18,
    max_parallel_jobs=3,
    objective_type="Maximize",
    objective_metric_name="eval_f1",
    metric_definitions=[
        {
            "Name": "eval_f1",
            "Regex": "eval_f1_weighted: ([0-9\\.]+)"
        }
    ],
    hyperparameter_ranges={
        "learning_rate": ContinuousParameter(1e-5, 1e-3),
        "weight_decay": ContinuousParameter(0.0, 0.1),
        "adam_beta1": ContinuousParameter(0.8, 0.999),
        "adam_beta2": ContinuousParameter(0.8, 0.999),
        "adam_epsilon": ContinuousParameter(1e-8, 1e-6),
        "label_smoothing_factor": ContinuousParameter(0.0, 0.1),
        "optim": CategoricalParameter(
            [
                "adamw_hf",
                "adamw_torch",
                "adamw_apex_fused",
                "adafactor"
            ]
        )
    }
)

In [ ]:
tuner.fit(inputs, wait=False)

params.sagemaker_tuning_job_name = tuner.latest_tuning_job.name

In [8]:
print(f"SageMaker tuning job name: {params.sagemaker_tuning_job_name}")

SageMaker tuning job name: pytorch-training-221209-0011


In [9]:
import pandas as pd

tuner_metrics: pd.DataFrame = sagemaker.HyperparameterTuningJobAnalytics(
    hyperparameter_tuning_job_name=params.sagemaker_tuning_job_name,
    sagemaker_session=sagemaker_session
).dataframe()

tuner_metrics["optim"] = tuner_metrics["optim"].apply(lambda x: x.strip('"'))

tuner_metrics.sort_values("FinalObjectiveValue", ascending=False, inplace=True)
tuner_metrics[["TrainingJobName", "FinalObjectiveValue", "TrainingJobStatus"]]

,TrainingJobName,FinalObjectiveValue,TrainingJobStatus
8,pytorch-training-221209-0011-010-d474d283,0.794327,Completed
6,pytorch-training-221209-0011-012-713a2cab,0.789217,Completed
1,pytorch-training-221209-0011-017-26c875ea,0.788787,Completed
13,pytorch-training-221209-0011-005-d9f53221,0.783829,Completed
0,pytorch-training-221209-0011-018-bbc1a967,0.782722,Completed
7,pytorch-training-221209-0011-011-0a4772df,0.780891,Completed
9,pytorch-training-221209-0011-009-67fb584b,0.780385,Completed
17,pytorch-training-221209-0011-001-ff4aab66,0.774016,Completed
12,pytorch-training-221209-0011-006-da6d3b9e,0.771442,Completed
5,pytorch-training-221209-0011-013-14b6c947,0.768171,Completed


Now, we can sort the results by the `FinalObjectiveValue` metric and see the best hyperparameters found.

In [10]:
best_job = tuner_metrics.iloc[0]
best_job.to_dict()

{'adam_beta1': 0.9944095815441554,
 'adam_beta2': 0.8750000522553327,
 'adam_epsilon': 1.8526084265228802e-07,
 'label_smoothing_factor': 0.047566123672759336,
 'learning_rate': 3.952388499692274e-05,
 'optim': 'adafactor',
 'weight_decay': 0.1,
 'TrainingJobName': 'pytorch-training-221209-0011-010-d474d283',
 'TrainingJobStatus': 'Completed',
 'FinalObjectiveValue': 0.794326901435852,
 'TrainingStartTime': Timestamp('2022-12-09 00:50:47-0300', tz='tzlocal()'),
 'TrainingEndTime': Timestamp('2022-12-09 01:07:13-0300', tz='tzlocal()'),
 'TrainingElapsedTimeSeconds': 986.0}

### Training best model

We will train the model using the best hyperparameters found.

We will concatenate the training and validation datasets to train the model with more data and evaluate it using the test dataset.

In [16]:
estimator.environment = {
    "MLFLOW_TRACKING_URI": params.mlflow_tracking_uri,
    "MLFLOW_EXPERIMENT_NAME": params.mlflow_experiment_name,
    "MLFLOW_TRACKING_USERNAME": params.mlflow_tracking_username,
    "MLFLOW_TRACKING_PASSWORD": params.mlflow_tracking_password,
    "MLFLOW_TAGS": params.mlflow_tags,
    "MLFLOW_RUN_ID": mlflow.active_run().info.run_id,
    "MLFLOW_FLATTEN_PARAMS": "True",
    "HF_MLFLOW_LOG_ARTIFACTS": "True",
    "HUGGINGFACE_HUB_TOKEN": params.huggingface_hub_token
}

estimator._hyperparameters = {
    "push_to_hub": "True",
    "hub_model_id": f"dougtrajano/{params.mlflow_experiment_name}",
    "num_train_epochs": params.num_train_epochs,
    "early_stopping_patience": params.early_stopping_patience,
    "batch_size": params.batch_size,
    "seed": params.seed,
    "concat_validation_set": "True",
    "eval_dataset": "test",
    "adam_beta1": best_job["adam_beta1"],
    "adam_beta2": best_job["adam_beta2"],
    "adam_epsilon": best_job["adam_epsilon"],
    "learning_rate": best_job["learning_rate"],
    "weight_decay": best_job["weight_decay"],
    "optim": best_job["optim"]
}

estimator.fit(inputs, wait=False)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: toxicity-target-type-identification-2023-02-26-12-48-26-617


In [17]:
remove_checkpoints(
    bucket_name=bucket_name,
    checkpoint_prefix=f"{prefix}/checkpoints",
    aws_profile_name=params.aws_profile_name
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Deleted 109 checkpoints from sagemaker-us-east-1-215993976552/ToChiquinho/toxicity-target-type-identification/checkpoints.


In [ ]:
mlflow.end_run()

## Documentation

- [Estimators — sagemaker documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)
- [HyperparameterTuner — sagemaker documentation](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html)
- [Configure and Launch a Hyperparameter Tuning Job - Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-ex-tuning-job.html)
- [Managed Spot Training in Amazon SageMaker - Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html)